<a href="https://colab.research.google.com/github/Ira-a02/Algorithm/blob/main/%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from collections import deque

In [34]:
def add_edge(graph, u, v, capacity):
    if u not in graph:
        graph[u] = {}
    graph[u][v] = capacity
    if v not in graph:
        graph[v] = {}
    graph[v][u] = 0  # Обратное ребро с нулевым capacity
    # {
    # 0: {2: 3},  ребро от источника (0) к работе (2) с емкостью 3
    # 2: {1: 2},  ребро от работы (2) к стоку (1) с емкостью 2
    # 1: {},      сток не имеет исходящих ребер
    # }

In [35]:
def find_in_width(graph, source, sink, parent):
    visited = set()
    queue = deque([source])
    visited.add(source)

    while queue:
        u = queue.popleft()
        for v in graph.get(u, {}):
            if v not in visited and graph[u][v] > 0:
                visited.add(v)
                parent[v] = u
                if v == sink:
                    return True
                queue.append(v)

    return False

In [36]:
def ford_falkerson(graph, source, sink):
    parent = {}
    max_flow = 0

    while find_in_width(graph, source, sink, parent):
        path_flow = float('Inf')
        s = sink

        while s != source:
            path_flow = min(path_flow, graph[parent[s]][s])
            s = parent[s]

        v = sink
        while v != source:
            u = parent[v]
            graph[u][v] -= path_flow
            graph[v][u] += path_flow
            v = parent[v]

        max_flow += path_flow

    return max_flow

In [68]:

def get_schedule(jobs, m, L):
    graph = {}
    source = 0
    sink = 1

    unique_time_points = set()
    for release, deadline, task_time in jobs:
        unique_time_points.add(release)
        unique_time_points.add(deadline)
    time_points = sorted(unique_time_points)

    intervals = []
    for i in range(len(time_points) - 1):
        intervals.append((time_points[i], time_points[i + 1]))

    for i, (release, deadline, task_time) in enumerate(jobs):
        add_edge(graph, source, 2 + i, task_time)  # Ребро от источника к работе с емкостью времени обработки

    for j, (start_time, end_time) in enumerate(intervals):
        add_edge(graph, 2 + len(jobs) + j, sink, m * (end_time - start_time))  # Ребро от интервала к стоку

    for i, (release, deadline, task_time) in enumerate(jobs):
        for j, (start_time, end_time) in enumerate(intervals):
            if release <= start_time and end_time <= deadline and start_time <= release + L :
                add_edge(graph, 2 + i, 2 + len(jobs) + j, end_time - start_time)  # Ребро от работ к интервалам с емкостью времени интервала

    flow_value = ford_falkerson(graph, source, sink)
    total_tasks_time = sum(map(lambda job: job[2], jobs))
    return flow_value == total_tasks_time

In [70]:
m = 2
L = 1

jobs = [(0, 5, 2), (1, 6, 1), (2, 5, 3)]# да
#jobs = [(0, 5, 2), (1, 6, 1), (2, 4, 3)]# нет
if get_schedule(jobs, m, L):
    print("Да, можно построить расписание.")
else:
    print("Нет, нельзя построить расписание.")

Нет, нельзя построить расписание.


In [29]:
#jobs = [(0, 5, 2),(1, 6, 1),(2, 4, 3),(3, 7, 2),(4, 8, 1),(5, 3, 4),(6, 9, 2),(7, 10, 3),(8, 11, 1),(9, 12, 2)]#с запаздыванием нет
#jobs = [(0, 5, 2),(1, 6, 1),(3, 7, 2),(4, 8, 1),(6, 9, 2),(8, 11, 1),(9, 12, 2)] # да